In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("E:\MagicBricks.csv")

df2=df.copy()


In [3]:
df2.isnull().sum()



Area             0
BHK              0
Bathroom         2
Furnishing       5
Locality         0
Parking         33
Price            0
Status           0
Transaction      0
Type             5
Per_Sqft       241
dtype: int64

In [4]:
df2.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Area         1259 non-null   float64
 1   BHK          1259 non-null   int64  
 2   Bathroom     1257 non-null   float64
 3   Furnishing   1254 non-null   object 
 4   Locality     1259 non-null   object 
 5   Parking      1226 non-null   float64
 6   Price        1259 non-null   int64  
 7   Status       1259 non-null   object 
 8   Transaction  1259 non-null   object 
 9   Type         1254 non-null   object 
 10  Per_Sqft     1018 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 108.3+ KB


In [5]:
df2['Area'] = df2['Area'].fillna(df2['Area'].mean())
df2['BHK'] = df2['BHK'].fillna(df2['BHK'].mean())
df2['Bathroom'] = df2['Bathroom'].fillna(df2['Bathroom'].mean())
df2['Per_Sqft'] = df2['Per_Sqft'].fillna(df2['Per_Sqft'].mean())

# Fill NaN values with the mode for categorical columns
df2['Furnishing'] = df2['Furnishing'].fillna(df2['Furnishing'].mode()[0])
df2['Locality'] = df2['Locality'].fillna(df2['Locality'].mode()[0])
df2['Parking'] = df2['Parking'].fillna(df2['Parking'].mode()[0])
df2['Price'] = df2['Price'].fillna(df2['Price'].mode()[0])
df2['Status'] = df2['Status'].fillna(df2['Status'].mode()[0])
df2['Transaction'] = df2['Transaction'].fillna(df2['Transaction'].mode()[0])
df2['Type'] = df2['Type'].fillna(df2['Type'].mode()[0])

df2.shape



(1259, 11)

In [6]:
df2.isnull().sum()


Area           0
BHK            0
Bathroom       0
Furnishing     0
Locality       0
Parking        0
Price          0
Status         0
Transaction    0
Type           0
Per_Sqft       0
dtype: int64

In [7]:
for i in ("Area","BHK","Bathroom","Parking","Price","Per_Sqft"):
    q1 = df2[i].quantile(0.25)
    q3= df2[i].quantile(0.75)
    IQR = q3 - q1
    lb = q1 - 1.5 * IQR
    ub = q3 + 1.5 * IQR
    df2 = df2[(df2[i] > lb) & (df2[i] < ub)]


print(df2.shape)
print(df2.shape)
print(df2.columns)


(1018, 11)
(1018, 11)
Index(['Area', 'BHK', 'Bathroom', 'Furnishing', 'Locality', 'Parking', 'Price',
       'Status', 'Transaction', 'Type', 'Per_Sqft'],
      dtype='object')


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd

# Assuming your data is in a pandas DataFrame named 'data'
X = df2.drop(['Price'],axis=1)
y = df2['Price']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps for different types of columns
numeric_features = ['Area', 'BHK', 'Bathroom', 'Parking', 'Per_Sqft']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_features = ['Furnishing', 'Locality', 'Status', 'Transaction', 'Type']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with preprocessing and the linear regression model
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

# Train the model
model.fit(X_train, y_train)
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [28]:
df2["Area"].unique()

array([  800.    ,   750.    ,   950.    ,   600.    ,   650.    ,
        1300.    ,  1350.    ,   985.    ,  1100.    ,   870.    ,
         630.    ,   660.    ,   344.4448,   550.    ,  1150.    ,
         850.    ,   900.    ,   430.556 ,   914.9315,   500.    ,
          28.    ,  2160.    ,  1500.    ,  1080.    ,  1450.    ,
         810.    ,  1710.    ,  1650.    ,  1280.    ,  1260.    ,
        1130.    ,  1000.    ,  2925.    ,  1600.    ,  1143.    ,
        8000.    ,   890.    ,  1387.    ,  1050.    ,  1200.    ,
         700.    ,  1700.    ,  1550.    ,  1400.    ,  1900.    ,
        1850.    ,  1800.    ,  1620.    ,  1540.    ,  2300.    ,
        1560.    ,   400.    ,  1525.    ,  2200.    ,  2880.    ,
        2700.    ,  1575.    ,  1675.    ,   825.    ,   875.    ,
         450.    ,   100.    ,    75.    ,   270.    ,   444.    ,
         729.    ,  4050.    ,  2400.    ,   405.    ,   666.    ,
        1360.    ,  2170.    ,   720.    ,  1030.    ,   350. 

In [59]:
import tkinter as tk
from tkinter.ttk import Combobox
import pandas as pd
import pickle

# Assuming model.pkl and data.xlsx are in the same directory as this script
# Load the trained model
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

# Load Excel data (replace 'data.xlsx' with your actual file name)
df2 = pd.read_csv("E:\MagicBricks.csv")

# Define unique values for dropdowns
areas = list(df2["Area"].unique())
bhks = list(df2['BHK'].unique())
bathrooms = list(df2['Bathroom'].unique())
furnishings = list(df2['Furnishing'].unique())
localities = list(df2['Locality'].unique())
parkings = list(df2['Parking'].unique())
statuses = list(df2['Status'].unique())
transactions = list(df2['Transaction'].unique())
types = list(df2['Type'].unique())
persqfts = list(df2['Per_Sqft'].unique())

def predict_price():
    area = area_var.get()
    bhk = int(bhk_var.get())
    bathroom = int(bathroom_var.get())
    furnishing = furnishing_var.get()
    locality = locality_var.get()
    parking = int(parking_var.get())
    status = status_var.get()
    transaction = transaction_var.get()
    type_ = type_var.get()
    persqft =int(persqft_var.get())
    
    input_data = pd.DataFrame([[area, bhk, bathroom, furnishing, locality, parking, status, transaction, type_, persqft]], 
                              columns=['Area', 'BHK', 'Bathroom', 'Furnishing', 'Locality', 'Parking', 'Status', 'Transaction', 'Type', 'Per_Sqft'])
    prediction = model.predict(input_data)[0]
    
    lbn.config(text=f"Predicted Price: {prediction:.2f}")

# Initialize Tkinter
root = tk.Tk()
root.title("House Price Prediction")


# Icon
head=Label(root,text="House Price Prediction",font="Arial 50",bg="sky blue")
head.place(x=0,y=0,width=1600)
img= tk.PhotoImage(file="C:\\Users\\91939\\Downloads\\newhouse.png")
myimage = tk.Label(root, image=img)
myimage.place(x=0, y=80,height=700,width=800)

# Entries Frame
entries = tk.Frame(root, width=600, height=600)
entries.place(x=800, y=200)

# Widgets
tk.Label(entries, text="Area", font="Arial 13").place(x=30, y=0)
area_var = tk.StringVar()
area_cmb = Combobox(entries, textvariable=area_var, values=areas)
area_cmb.place(x=30, y=30)

tk.Label(entries, text="BHK", font="Arial 13").place(x=400, y=0)
bhk_var = tk.StringVar()
bhk_cmb = Combobox(entries, textvariable=bhk_var, values=bhks)
bhk_cmb.place(x=400, y=30)

tk.Label(entries, text="Bathroom", font="Arial 13").place(x=30, y=60)
bathroom_var = tk.StringVar()
bathroom_cmb = Combobox(entries, textvariable=bathroom_var, values=bathrooms)
bathroom_cmb.place(x=30, y=90)

tk.Label(entries, text="Furnishing", font="Arial 13").place(x=400, y=60)
furnishing_var = tk.StringVar()
furnishing_cmb = Combobox(entries, textvariable=furnishing_var, values=furnishings)
furnishing_cmb.place(x=400, y=90)

tk.Label(entries, text="Locality", font="Arial 13").place(x=30, y=120)
locality_var = tk.StringVar()
locality_cmb = Combobox(entries, textvariable=locality_var, values=localities)
locality_cmb.place(x=30, y=150)

tk.Label(entries, text="Parking", font="Arial 13").place(x=400, y=120)
parking_var = tk.StringVar()
parking_cmb = Combobox(entries, textvariable=parking_var, values=parkings)
parking_cmb.place(x=400, y=150)

tk.Label(entries, text="Status", font="Arial 13").place(x=30, y=180)
status_var = tk.StringVar()
status_cmb = Combobox(entries, textvariable=status_var, values=statuses)
status_cmb.place(x=30, y=210)

tk.Label(entries, text="Transaction", font="Arial 13").place(x=400, y=180)
transaction_var = tk.StringVar()
transaction_cmb = Combobox(entries, textvariable=transaction_var, values=transactions)
transaction_cmb.place(x=400, y=210)

tk.Label(entries, text="Type", font="Arial 13").place(x=30, y=240)
type_var = tk.StringVar()
type_cmb = Combobox(entries, textvariable=type_var, values=types)
type_cmb.place(x=30, y=270)

tk.Label(entries, text="Per Sqft", font="Arial 13").place(x=400, y=240)
persqft_var = tk.StringVar()
persqft_cmb = Combobox(entries, textvariable=persqft_var, values=persqfts)
persqft_cmb.place(x=400, y=270)

# Prediction Label
lbn = tk.Label(entries, text="", font="Arial 20")
lbn.place(x=100, y=350)

# Predict Button
btn = tk.Button(entries, text="PREDICT", bg="skyblue", command=predict_price)
btn.place(x=200, y=400, width=200, height=50)

# Run the GUI
root.mainloop()

['Rohini Sector 25',
 'J R Designers Floors, Rohini Sector 24',
 'Citizen Apartment, Rohini Sector 13',
 'Rohini Sector 24',
 'Rohini Sector 24 carpet area 650 sqft ',
 'Delhi Homes, Rohini Sector 24',
 'Rohini Sector 21',
 'Rohini Sector 22',
 'Rohini Sector 20',
 'Rohini Sector 8 ',
 'Rohini Sector 25 carpet area 660 sqft ',
 'Rohini Sector 16',
 'Rohini Sector 14',
 'Mayur Apartment, Rohini Sector 9',
 'Vijay Vihar, Rohini',
 'Rohini Sector 18 carpet area 40 sqm ',
 'Prashant Vihar, Rohini',
 'Rohini Sector 5',
 'Sanskriti Apartment, Rohini Sector 28',
 'Rohini Sector 19',
 'Lajpat Nagar 2',
 'Lajpat Nagar 1',
 'Lajpat Nagar',
 'Vikram Vihar, Lajpat Nagar',
 'Lajpat Nagar 4',
 'Lajpat Nagar 3',
 'Dayanand Colony, Lajpat Nagar ',
 'Amar Colony, Lajpat Nagar',
 'DDA Flats Sector 14, Dwarka Sector 14',
 'Siddhartha Heights, Dwarka Sector 24',
 'MTNL Employees House Welfare Society, Dwarka Sector 24',
 'Delhi Homes Yojana, Dwarka Sector 24',
 'The Elite Spire, Dwarka Sector 21',
 'Kunj 